In [ ]:

import json

def parse_ikat_data(data_path):

    with open(data_path, 'r') as f:
        topics = json.load(f)

    ptkb_turn = {}
    utterance_turn = {}
    response_turn = {}

    for topic in topics:
        topic_id = topic['number']
        for turn in topic['turns']:
            turn_num = str(topic_id) + '_' + str(turn['turn_id'])
            if 'response' in turn:
                utterance_turn[turn_num] = turn['resolved_utterance']
                response_turn[turn_num] = turn['response']
                relevant_statements = [topic['ptkb'][str(i)] for i in turn['ptkb_provenance']]
                if len(relevant_statements)>0:
                    ptkb_turn[turn_num] = ' '.join(relevant_statements).strip()
                    print(ptkb_turn[turn_num])
                else:
                    ptkb_turn[turn_num] = 'NONE'

                print(ptkb_turn[turn_num])
                print(utterance_turn[turn_num])
                print(turn_num)
                print('*********************')
            else:
                print('topic ended at this point: ', turn_num)
                break
    return  utterance_turn, ptkb_turn  , response_turn         


prompt = """instruction: {inst}\n
Background info about user: {bkg}\n
User question: {question}\n
document: {doc}\n
Please only generate an int score between 0 to 4 to say to what extent the document is relevant to the user question. Score lower than 2 means the document is not relevant.
"""

instruction_3 = """I will give you a user question and a passage, you should say to what extent the given passage is relevant for answering the question by giving an integer rate between 0-4. I will also give some background information about the user and history of the user's conversation with the system. """

data_path = 'data/2023_test_topics.json'

utterance_turn, ptkb_turn, response_turn = parse_ikat_data(data_path)


In [42]:
def clean(passage_text):
    return passage_text.replace('\t', ' ').replace('\n', ' ').replace('  ', ' ').strip()


In [ ]:
import pickle

qrel_path = 'pools/human_qrels'
with open(qrel_path, 'r') as f:
    lines = f.readlines()

with open('data/passage_collection.pkl', 'rb') as f:
    passage_collection = pickle.load(f)

lines_to_write = []
lines_to_write.append('turn_id\trewritten_utterance\tpassage_id\tpassage_text\tscore\tptkb\tNone\n')

for line in lines:
    turn_id, _, passage_id, score = line.split(' ')    
    score = score.strip()
    tmp_line = turn_id +'\t'+ utterance_turn[turn_id] + '\t'+ response_turn[turn_id] + '\t' + passage_id +'\t'+  clean(passage_collection[passage_id]) +'\t' +  score  +'\t' + ptkb_turn[turn_id] + '\t0\n'
    print(passage_collection[passage_id])
    lines_to_write.append(tmp_line)


In [44]:
with open('data.txt', 'w') as f:
    f.writelines(lines_to_write)

In [45]:
import pickle

qrel_path = 'pools/human_qrels'
with open(qrel_path, 'r') as f:
    lines = f.readlines()

with open('data/passage_collection.pkl', 'rb') as f:
    passage_collection = pickle.load(f)

lines_to_write = []
lines_to_write.append('turn_id\trewritten_utterance\tpassage_id\tpassage_text\tscore\tptkb\tNone\n')

from collections import defaultdict
turn_score_passage = defaultdict(lambda: defaultdict(list))

for line in lines:
    turn_id, _, passage_id, score = line.split(' ')    
    score = score.strip()
    turn_score_passage[turn_id][score].append(passage_id)



In [61]:
import random

turn_positive_passage = {}
turn_negative_passage = {}

for turn_id in turn_score_passage:

    for key in ['4', '3', '2']:
        if len(turn_score_passage[turn_id][key])>0:
            pass_id = random.choice(turn_score_passage[turn_id][key])
            turn_positive_passage[turn_id] = " ".join(passage_collection[pass_id].split()).strip()+'\t'+key
            break

    for key in ['0', '1']:
        if len(turn_score_passage[turn_id][key])>0:
            pass_id = pass_id = random.choice(turn_score_passage[turn_id][key])
            turn_negative_passage[turn_id] = " ".join(passage_collection[pass_id].split()).strip()+'\t'+key
            break
        

In [ ]:
for turn_id in turn_negative_passage:
    print(turn_negative_passage[turn_id])
    print(turn_positive_passage[turn_id])
    print('*********************')

In [66]:
lines = []
path_1 = 'data/random_positive_negative_examples.txt'

for turn_id in turn_negative_passage:
    x = turn_id + '\t' + turn_negative_passage[turn_id] + '\t' + turn_positive_passage[turn_id] + '\t0\n'
    lines.append(x)

    
with open(path_1, 'w') as f:
    f.writelines(lines)